In [120]:
import pandas as pd
import re
import numpy as np



In [121]:

df_2 = pd.read_excel(r'C:\Users\mario.lira\Documents\BASE PARA TESTE\BASES\Cópia de BASE POR OPERADORA (1).xlsx', sheet_name='Up Health')
df_1 = pd.read_excel(r'C:\Users\mario.lira\Documents\BASE PARA TESTE\BASES\Extrato Up.xlsx')

In [122]:

#FUNCAO QUE TRATA OS DADOS E FAZ O MERGE DAS BASES

def only_digits(s):
    return re.sub(r"\D", "", str(s) if s is not None else "")
def trataremerge(df_1,df_2):

        print("Fazendo o tratamento dos dados")
        try:
                    df_1 = df_1.copy()
                    df_1["CPF_CNPJ_TIT_NUM"] = (
                        df_1["CPF/CNPJ Titular"]
                        .map(only_digits)                     # só dígitos
                        .replace({"": None})                  # vazio -> NaN
                        .pipe(pd.to_numeric, errors="coerce") # para número (float -> depois Int64)
                        .astype("Int64")                      # inteiro nulo do pandas
                    )


                    # df_2
                    df_2 = df_2.copy()
                    df_2["CNPJ_NUM"] = (
                        df_2["CNPJ"]
                        .astype(str)
                        .str.replace(r"\D", "", regex=True)
                        .replace({"": None})
                        .pipe(pd.to_numeric, errors="coerce")
                        .astype("Int64")
                    )

                    print("Fazendo o merge dos dados")

                    visao = pd.merge(df_1,df_2, 
                    left_on = "CPF/CNPJ Titular", right_on= 'CNPJ', how="outer")
                    print("Fazendo o merge dos dados")
        except Exception as e:
                        print("Erro ao fazer o merge dos dados da uma olhada la: ", e)
                        visao = pd.DataFrame()
        return visao

In [123]:

retorno = trataremerge(df_1, df_2)
retorno

Fazendo o tratamento dos dados
Fazendo o merge dos dados
Fazendo o merge dos dados


,Porte Contrato,Contrato,Operadora,Marca Ótica,Migrado,Contrato Operadora,Proposta,CPF/CNPJ Titular,Nome/Empresa Titular,Tipo Venda,...,Faixa etária,CPF_CNPJ_TIT_NUM,OPERADORA,CNPJ,EMPRESA,SUB-ESTIPULANTE,CONSULTOR,ADICIONADO MOLTRIO,GESTÃO COMERCIAL,CNPJ_NUM
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,UP HEALTH,5.254813e+09,GABRIEL FERNANDES AZEVEDO,FABIANA MOTTA RIBEIRO (NV),FABIANA MOTTA RIBEIRO (NV),SIM,NaN,5254813202
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,UP HEALTH,5.428676e+09,MARIA EDUARDA ALVARINDO FERREIRA,FABIANA MOTTA RIBEIRO (NV),FABIANA MOTTA RIBEIRO (NV),SIM,NaN,5428675594
2,Adesão,09500 - CRF - CONS REG DE FARMACIA DO ES,SAO BERNARDO SAMP,1004957232,N,28167666000158,9.000013e+09,5.893018e+09,PATRICIA LORRANY CELESTINO,Venda Online,...,de 34 até 38 anos,58930177220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,Adesão,07957 - FETRACS - FED DOS TRAB NO COMERCIO - V...,SAO BERNARDO SAMP,1004832727,N,26280133000152,9.000016e+09,5.945133e+09,ELIZAMA SILVA DE JESUS,Venda Online,...,de 29 até 33 anos,59451325190,UPHEALTH,5.945133e+09,ELIZAMA SILVA DE JESUS,FABIANA MOTTA RIBEIRO (NV),FABIANA MOTTA RIBEIRO (NV),NÃO,NaN,5945132519
4,Adesão,14244 - ASPROLI - VIG 01,SAO BERNARDO SAMP,1004738575,N,11490989000185,9.000015e+09,7.917194e+09,LARISSA PACHECO BERNARDI,Venda Online,...,de 44 até 48 anos,79171937560,UP HEALTH,7.917194e+09,LARISSA PACHECO BERNARDI,FABIANA MOTTA RIBEIRO (NV),FABIANA MOTTA RIBEIRO (NV),SIM,NaN,7917193756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35838,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,373458970,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,de 29 até 33 anos,975539650001430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
35839,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,372472336,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,de 39 até 43 anos,975539650001430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
35840,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,371814832,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,de 44 até 48 anos,975539650001430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
35841,Adesão,20000 - CAEEPP - SAMP VIG 01,SAO BERNARDO SAMP,NaN,N,24376076000184,NaN,NaN,NaN,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [124]:
retorno.to_excel("primeiro_teste.xlsx", index=False)

In [125]:
# FUNCAO QUE APLICA AS REGRAS DE TIPO DE PRODUTO BASEDO NO NOME DO PLANO
def regras_produto(retorno):

        # --- normaliza para comparar com segurança
        tipo_norm = retorno['Tipo Produto'].astype(str).str.strip().str.casefold()
        prod_norm = retorno['Nome plano'].astype(str).str.strip().str.casefold()

        # só mexe nas linhas onde o tipo atual é "ambos"
        mask_ambos = tipo_norm.eq('ambos')

        # conjuntos de produtos -> novo tipo
        produtos_saude  = {
            'sao bernardo way especial qc',
            'sao bernardo way executivo qp',
        }
        produtos_odonto = {
            'odonto sb essencial empresrial',
            'ODONTO SB ESSENCIAL EMPRESRIAL'  # ajuste o texto conforme sua base
        }

        # aplica as regras
        retorno.loc[mask_ambos & prod_norm.isin(produtos_saude),  'Tipo Produto'] = 'Médico'
        retorno.loc[mask_ambos & prod_norm.isin(produtos_odonto), 'Tipo Produto'] = 'Dental'
        print("Regras de produto aplicadas com sucesso")
        return retorno

In [126]:
retorno = regras_produto(retorno)

Regras de produto aplicadas com sucesso


In [127]:
retorno.to_excel("segundo_teste.xlsx", index=False)

In [128]:
# FUNCAO QUE SEPARA AS BASES DE SAUDE E ODONTO
def separa_bases(retorno):
    col_op = "Tipo Produto"  # confirme o nome exato
    op_norm = retorno[col_op].astype(str).str.strip().str.casefold()

    # compare com minúsculas, pois usamos casefold()
    mask_medico = op_norm.eq("médico") | op_norm.eq("medico")
    mask_dental = op_norm.eq("dental")

    base_medico = retorno[mask_medico].copy()
    base_dental = retorno[mask_dental].copy()
    return base_medico, base_dental

base_saude, base_odonto = separa_bases(retorno)

with pd.ExcelWriter("saida.xlsx", engine="openpyxl") as xlw:
    base_saude.to_excel(xlw, sheet_name="Médico", index=False)
    base_odonto.to_excel(xlw, sheet_name="Dental", index=False)

In [129]:
sindicatos_df = pd.read_excel(r'C:\Users\mario.lira\Documents\BASE PARA TESTE\BASES\Sindicatos.xlsx')

In [130]:
# FUNCAO QUE FAZ O MERGE DOS SINDICATOS
def merge_sindicatos(base_saude, base_odonto, sindicatos_df):
    print("Fazendo o mapeamento dos sindicatos")

    mapas_sindicatos = {
        'ACE BSI COMPULSORIO' : 'ACE-BSI',
        'ACE BSI LIVRE ADESAO': 'ACE-BSI',
        'ACE BSI-COMP-C-SBS-N VENDA-PME': 'ACE-BSI',
        'ACE BSI-COMP-S-SAMP-N VENDAS-PME': 'ACE-BSI',
        'ACE BSI-FACUL-C-SBS-N VENDA-PME': 'ACE-BSI',
        'ACE BSI-FACUL-S-SAMP-N VENDAS-PME ': 'ACE-BSI',
        'ACE BSI-O-SAMP-N VENDAS-SAMP': 'ACE-BSI',
        'Astic Linear - Compulsório': 'ASTIC',
        'ASTIC-COMP-S-SAMP-N. VENDAS-PME': 'ASTIC',
        'ASTIC-COMP-S-SAMP-TOMB-PME': 'ASTIC',
        'ASTIC-FACUL-S-SAMP-N VENDAS-PME': 'ASTIC',
        'ASTIC-FACUL-S-SAMP-TOMB-PME': 'ASTIC',
        'SINDICOMERCIARIOS COMPULSORIO' :'SINDICOMERCIÁRIOS',
        'SINDICOMERCIARIOS LIVRE ADESÃO':'SINDICOMERCIÁRIOS',
        'SINDICOMERCIÁRIOS MAIS SAUDE':'SINDICOMERCIÁRIOS',
        'SINDICOMERCIARIOS-COMP-S-SAMP-N VENDAS-PME':'SINDICOMERCIÁRIOS',
        'SINDICOMERCIARIOS-FACUL-S-SAMP-N VENDAS-PME':'SINDICOMERCIÁRIOS',
        'SINDICOMERCIARIOS-O-SAMP-N VENDAS-PME':'SINDICOMERCIÁRIOS',
        'SINDUSCON - ODONTO' : 'SINDUSCON',
        'SINDUSCON 2023 - COMPULSORIO' : 'SINDUSCON',
        'ABEEP - SAO BERNARDO':'ABEEPP',
        'ABEEPP ':'ABEEPP',
        'ABEEPP SBS COMBO ADESÃO':'ABEEPP',
        'ABES-TAO-SBS SAMP-N.VENDAS-CA' : 'ABES',
        'ADEPOL - SBS COMBO - SAÚDE E ODONTO' : 'ADEPOL',
        'SINTRAFARMA COMPULSORIO' : 'SINTRAFARMA',
        'SINTRAFARMA-COMP-S-SAMP-N VENDAS-PME' : 'SINTRAFARMA',
        'SINTRAFARMA-FACUL-S-SAMP-N VENDA-PME' : 'SINTRAFARMA',
        'SINTRAFARMA-O-SAMP-N VENDAS-PME' : 'SINTRAFARMA',
        'SINTRAMASSAS-COMP-S-SAMP-N VENDAS-PME': 'SINTRAMASSAS',
        'SINTRAMASSAS-COMP-S-SAMP-N VENDAS-PME-PART': 'SINTRAMASSAS',
        'SINTRAMASSAS-O-SAMP-N VENDAS-PME': 'SINTRAMASSAS',
        'FETRACS - ADESAO' : 'FETRACS',
        'FETRACS - SAO BERNARDO' : 'FETRACS',
        'FETRACS-COMP-S-SAMP-N VENDAS-PME': 'FETRACS',
        'SINDEPRES COMPULSORIO' : 'SINDEPRES',
        'SINDEPRES-COMP-S-SAMP-N VENDA-PME': 'SINDEPRES',
        'CAEBS-COMP-C-SBS-N VENDA-PME': 'CAEBS',
        'CAEBS-COMP-S-SAMP-N VENDAS-PME': 'CAEBS',
        'CAEBS-O-SAMP-N VENDAS-PME': 'CAEBS',
        'ABOES': 'ABOES',
        'AMAPES': 'AMAPES',
        'ASPROLI  ': 'ASPROLI',
        'ASSEMES SB COMBO ADESÃO': 'ASSEMES',
        'CAEEPP SAMP':'CAEEPP',
        'CRC ES': 'CRC ES',
        'CRFES': 'CRF ES',
        'SINDIENFERMEIROS - ADESAO': 'SINDIENFERMEIROS',
        'SINDILIMPE-COMP-S-SAMP-N VENDAS-PME':'SINDILIMPE',
        'SINTRACON-COMP-S-SAMP-N VENDAS-PME':'SINTRACON',
        'UP HEALTH - ODONTO DIRETOS': 'UP HEALTH - ODONTO DIRETOS',
    }

    # mapeia o texto do "Grupo gerencial" para o nome do sindicato
    for df in (base_saude, base_odonto):
        df['Sindicatos'] = df['Grupo gerencial'].map(mapas_sindicatos)

    print("Fazendo o merge dos sindicatos")
    out_saude = base_saude.merge(sindicatos_df, left_on='Sindicatos', right_on='SINDICATO', how='inner')
    out_odonto = base_odonto.merge(sindicatos_df, left_on='Sindicatos', right_on='SINDICATO', how='inner')

    print('Tudo certo com o merge dos sindicatos acabei aqui')
    return out_saude, out_odonto


In [131]:
base_saude, base_odonto = merge_sindicatos(base_saude, base_odonto, sindicatos_df)

Fazendo o mapeamento dos sindicatos
Fazendo o merge dos sindicatos
Tudo certo com o merge dos sindicatos acabei aqui


In [132]:
base_saude

,Porte Contrato,Contrato,Operadora,Marca Ótica,Migrado,Contrato Operadora,Proposta,CPF/CNPJ Titular,Nome/Empresa Titular,Tipo Venda,...,EMPRESA,SUB-ESTIPULANTE,CONSULTOR,ADICIONADO MOLTRIO,GESTÃO COMERCIAL,CNPJ_NUM,Sindicatos,SINDICATO,ÍNDICE,DESCRIÇÃO
0,Adesão,09500 - CRF - CONS REG DE FARMACIA DO ES,SAO BERNARDO SAMP,1004957232,N,28167666000158,9.000013e+09,5.893018e+09,PATRICIA LORRANY CELESTINO,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,CRF ES,CRF ES,0.4651,CONSELHO REGIONAL DE FARMÁCIA DO ESTADO DO ESP...
1,Adesão,07957 - FETRACS - FED DOS TRAB NO COMERCIO - V...,SAO BERNARDO SAMP,1004832727,N,26280133000152,9.000016e+09,5.945133e+09,ELIZAMA SILVA DE JESUS,Venda Online,...,ELIZAMA SILVA DE JESUS,FABIANA MOTTA RIBEIRO (NV),FABIANA MOTTA RIBEIRO (NV),NÃO,NaN,5945132519,FETRACS,FETRACS,0.4651,FEDERAÇÃO DOS TRABALHADORES NO COMÉRCIO E SERV...
2,Adesão,15540 - FETRACS - SAO BERNARDO - VIG 01,SAO BERNARDO SAUDE,994795724,N,26280133000152,9.000016e+09,8.525923e+09,JEANE BRIDI RAMOS,Venda Online,...,JEANE BRIDI RAMOS,BRUNO RAMOS BALLISTA (NV),BRUNO RAMOS BALLISTA (NV),SIM,NaN,8525922722,FETRACS,FETRACS,0.4651,FEDERAÇÃO DOS TRABALHADORES NO COMÉRCIO E SERV...
3,Adesão,17668 - ABEEPP - SAO BERNARDO VIG 01,SAO BERNARDO SAUDE,994437348,N,29025076000153,9.000010e+09,8.729133e+09,LIVIA DA CONCEICAO LOPES,Venda Online,...,LIVIA DA CONCEICAO LOPES,(A VERIFICAR) - DIONATA SILVA FERREIRA,NaN,SIM,NaN,8729132797,ABEEPP,ABEEPP,0.4651,ASSOCIACAO BRASILEIRA DOS ESTUDANTES DE ESCOLA...
4,Adesão,17711 - ASSEMES - SAO BERNARDO VIG 20,SAO BERNARDO SAUDE,994889852,N,15752539000174,9.000010e+09,8.895570e+09,ALESSANDRA DOS SANTOS,Venda Online,...,ALESSANDRA DOS SANTOS,PETERSON GONÇALVES,(ILHA) PETERSON GONÇALVES RIBEIRO,NaN,NaN,8895569709,ASSEMES,ASSEMES,0.4651,ASSOCIAÇÃO DOS SERVIDORES PÚBLICOS NO ESTADO D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30502,PME I,03851 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,372348520,N,3738466369,NaN,9.755372e+13,LOCSUL COMERCIO E LOCACOES LTDA ME,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30503,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,373986452,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30504,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,373458970,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30505,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,372472336,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...


In [133]:
base_saude.to_excel("base_saude2.xlsx", index=False)

In [134]:
base_odonto

,Porte Contrato,Contrato,Operadora,Marca Ótica,Migrado,Contrato Operadora,Proposta,CPF/CNPJ Titular,Nome/Empresa Titular,Tipo Venda,...,EMPRESA,SUB-ESTIPULANTE,CONSULTOR,ADICIONADO MOLTRIO,GESTÃO COMERCIAL,CNPJ_NUM,Sindicatos,SINDICATO,ÍNDICE,DESCRIÇÃO
0,Adesão,08082 - ABO ES SUCURSAL - ASSOC BRAS ODONT DO ...,SAO BERNARDO SAMP,1004331519,N,27242833000204,9.000009e+09,1.618867e+10,DANIELLE RODRIGUES FROEDE,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,ABOES,ABOES,0.4651,ASSOCIAÇÃO BRASILEIRA DE ODONTOLOGIA – SEÇÃO D...
1,PME I,16800 - JOAO BATISTA PIMENTA,SAO BERNARDO SAMP,1004600250,N,264004000187,9.000004e+09,2.640040e+11,JOAO BATISTA PIMENTA,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
2,PME I,13125 - YARA ALIMENTOS LTDA,SAO BERNARDO SAMP,1005010340,N,277552000737,5.471000e+03,2.775520e+11,YARA ALIMENTOS LTDA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
3,PME I,13125 - YARA ALIMENTOS LTDA,SAO BERNARDO SAMP,1004027736,N,277552000737,5.471000e+03,2.775520e+11,YARA ALIMENTOS LTDA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
4,PME I,13125 - YARA ALIMENTOS LTDA,SAO BERNARDO SAMP,1003916131,N,277552000737,5.471000e+03,2.775520e+11,YARA ALIMENTOS LTDA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,PME II,22172 - BRITANIA ELETRODOMESTICOS SA,SAO BERNARDO SAMP,1004616140,N,76492701001803,7.649270e+07,7.649270e+13,BRITANIA ELETRODOMESTICOS SA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
4976,PME II,22172 - BRITANIA ELETRODOMESTICOS SA,SAO BERNARDO SAMP,1004616147,N,76492701001803,7.649270e+07,7.649270e+13,BRITANIA ELETRODOMESTICOS SA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
4977,PME II,22172 - BRITANIA ELETRODOMESTICOS SA,SAO BERNARDO SAMP,1004616075,N,76492701001803,7.649270e+07,7.649270e+13,BRITANIA ELETRODOMESTICOS SA,Venda Nova,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
4978,PME I,24475 - COMERCIO DE BORDADOS RENDAS E TECIDOS ...,SAO BERNARDO SAMP,1004771173,N,9000015440,9.000015e+09,9.617467e+13,COMERCIO DE BORDADOS RENDAS E TECIDOS CURY LTDA,Venda Online,...,COMERCIO DE BORDADOS RENDAS E TECIDOS CURY LTDA,ILHA AZUL - RECUPERAÇÃO,ILHA AZUL - RECUPERAÇÃO,SIM,SIM,96174669000179,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...


In [135]:
# FUNCAO QUE DESCONSIDERA OS SUBSTIPULANTES INDESEJADOS
def desconsiderar_substipulantes(out_saude, base_odonto):
    print("Desconsiderando os substipulantes indesejados")
    substipulante_descosideradas={'(CO-CORRETORA) ILHA SERRANA','(CO-CORRETORA) CAPIXABAS','(CO-CORRETORA) C.S. CORRETORA','(CO-CORRETORA) ONDA AZUL','(CO-CORRETORA) ILHA SERRANA','(CO-CORRETORA) ONDA AZUL','(CO-CORRETORA) HAESE COLATINA','ILHA AZUL - RECUPERAÇÃO','(CO-CORRETORA) HAESE & LOZER','PÓS VENDAS - RECUPERAÇÃO'}

    mask_desconsiderar = out_saude['SUB-ESTIPULANTE'].isin(substipulante_descosideradas)
    mask_desconsiderar2 = base_odonto['SUB-ESTIPULANTE'].isin(substipulante_descosideradas)


    base_saude_mask = out_saude[~mask_desconsiderar].copy()
    base_odonto_mask = base_odonto[~mask_desconsiderar2].copy()
    return base_saude_mask, base_odonto_mask



In [136]:
desconsiderar_substipulantes(base_saude, base_odonto)
base_saude_mask, base_odonto_mask = desconsiderar_substipulantes(base_saude, base_odonto)

Desconsiderando os substipulantes indesejados
Desconsiderando os substipulantes indesejados


In [137]:
competencia='31/07/2024'

In [138]:
base_sd = base_saude_mask[base_saude_mask['Data Início Vigência'] <= pd.to_datetime(competencia, format='%d/%m/%Y', errors='coerce')]   

In [139]:
base_od = base_odonto_mask[base_odonto_mask['Data Início Vigência'] <= pd.to_datetime(competencia, format='%d/%m/%Y', errors='coerce')]

In [140]:
base_sd

,Porte Contrato,Contrato,Operadora,Marca Ótica,Migrado,Contrato Operadora,Proposta,CPF/CNPJ Titular,Nome/Empresa Titular,Tipo Venda,...,EMPRESA,SUB-ESTIPULANTE,CONSULTOR,ADICIONADO MOLTRIO,GESTÃO COMERCIAL,CNPJ_NUM,Sindicatos,SINDICATO,ÍNDICE,DESCRIÇÃO
4,Adesão,17711 - ASSEMES - SAO BERNARDO VIG 20,SAO BERNARDO SAUDE,994889852,N,15752539000174,9.000010e+09,8.895570e+09,ALESSANDRA DOS SANTOS,Venda Online,...,ALESSANDRA DOS SANTOS,PETERSON GONÇALVES,(ILHA) PETERSON GONÇALVES RIBEIRO,NaN,NaN,8895569709,ASSEMES,ASSEMES,0.4651,ASSOCIAÇÃO DOS SERVIDORES PÚBLICOS NO ESTADO D...
9,Adesão,14008 - SINDIENFERMEIROS - SIND ENFERMEIROS NO...,SAO BERNARDO SAMP,1003976648,N,30778641000132,9.000005e+09,9.558169e+09,GABRIELA FRECHIANI MARTINS DE SOUZA,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDIENFERMEIROS,SINDIENFERMEIROS,0.4651,SINDICATO DOS ENFERMEIROS NO ESTADO DO ESPÍRIT...
10,Adesão,14008 - SINDIENFERMEIROS - SIND ENFERMEIROS NO...,SAO BERNARDO SAMP,1003976648,N,30778641000132,9.000005e+09,9.558169e+09,GABRIELA FRECHIANI MARTINS DE SOUZA,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDIENFERMEIROS,SINDIENFERMEIROS,0.4651,SINDICATO DOS ENFERMEIROS NO ESTADO DO ESPÍRIT...
11,Adesão,14008 - SINDIENFERMEIROS - SIND ENFERMEIROS NO...,SAO BERNARDO SAMP,1003976648,N,30778641000132,9.000005e+09,9.558169e+09,GABRIELA FRECHIANI MARTINS DE SOUZA,Venda Online,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDIENFERMEIROS,SINDIENFERMEIROS,0.4651,SINDICATO DOS ENFERMEIROS NO ESTADO DO ESPÍRIT...
13,Adesão,00088 - ABEEPP - ADESÃO - VIG 01 - VENC 01,SAO BERNARDO SAMP,1004397983,N,100000182233,9.000010e+09,1.174346e+10,JANAINA BATISTA CAMARGO,Venda Online,...,JANAINA BATISTA CAMARGO,LEONARA LUIZ SILVA,(ILHA) DIONATAN SILVA FERREIRA,NaN,NaN,11743456751,ABEEPP,ABEEPP,0.4651,ASSOCIACAO BRASILEIRA DOS ESTUDANTES DE ESCOLA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30502,PME I,03851 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,372348520,N,3738466369,NaN,9.755372e+13,LOCSUL COMERCIO E LOCACOES LTDA ME,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30503,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,373986452,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30504,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,373458970,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...
30505,PME I,02396 - SINDICOMERCIARIOS,SAO BERNARDO SAMP,372472336,N,3738440554,NaN,9.755397e+13,PACREAUTO DISTRIBUIDORA LTDA,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,SINDICOMERCIÁRIOS,SINDICOMERCIÁRIOS,0.1328,SINDICATO DOS EMPREGADOS NO COMÉRCIO DO ESTADO...


In [141]:
base_sd.to_excel("base_saude_final.xlsx", index=False)

In [142]:
agrupado_sd = (base_sd
            .groupby(['SINDICATO','Grupo gerencial','Percentual','Função','ÍNDICE'],
                     as_index=False, dropna=False)['Valor']
            .sum());
     


In [143]:
agrupado_od = (base_od
            .groupby(['SINDICATO','Grupo gerencial','Percentual','Função','ÍNDICE'],
                     as_index=False, dropna=False)['Valor']
            .sum())

In [144]:
agrupado_sd.to_excel("agrupado_saude.xlsx", index=False)

In [145]:
agrupado_od.to_excel("agrupado_od.xlsx", index=False)


In [146]:
agrupado_od

,SINDICATO,Grupo gerencial,Percentual,Função,ÍNDICE,Valor
0,ABOES,ABOES,2.00,Vitalício Líder,0.4651,6.35
1,ACE-BSI,ACE BSI-COMP-C-SBS-N VENDA-PME,3.60,Vitalício,0.1328,2.38
2,ACE-BSI,ACE BSI-FACUL-C-SBS-N VENDA-PME,3.60,Vitalício,0.1328,0.68
3,ACE-BSI,ACE BSI-O-SAMP-N VENDAS-SAMP,10.00,Vitalício,0.1328,79.21
4,ACE-BSI,ACE BSI-O-SAMP-N VENDAS-SAMP,10.00,Vitalício sobre Movimentação,0.1328,61.19
5,CAEBS,CAEBS-COMP-C-SBS-N VENDA-PME,3.60,Vitalício,0.4651,0.17
6,SINDICOMERCIÁRIOS,SINDICOMERCIARIOS-O-SAMP-N VENDAS-PME,0.46,Vitalício Líder,0.1328,174.66
7,SINDICOMERCIÁRIOS,SINDICOMERCIARIOS-O-SAMP-N VENDAS-PME,10.00,Vitalício,0.1328,451.51
8,SINDICOMERCIÁRIOS,SINDICOMERCIARIOS-O-SAMP-N VENDAS-PME,10.00,Vitalício sobre Movimentação,0.1328,381.32
9,SINDUSCON,SINDUSCON - ODONTO,10.00,Vitalício,0.1328,9.42
